In [1]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 24.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

def tiff_to_gdf(tiff_path):
    # Open the GeoTIFF file using rasterio
    with rasterio.open(tiff_path) as src:
        # Get the CRS of the raster (spatial reference)
        crs = src.crs
        band_names = src.descriptions
        # Read the bands into an array (you can modify this to get specific bands if needed)
        bands = []
        for i in range(1, src.count + 1):
            band_data = src.read(i)  # Read each band
            bands.append(band_data.flatten())  # Flatten to make it 1D

        # Combine all bands into a DataFrame
        band_df = pd.DataFrame(np.array(bands).T, columns=band_names)

        # Generate the coordinates for each pixel
        rows, cols = np.indices(band_data.shape)  # Indices for each pixel
        x, y = src.xy(rows, cols)  # Convert indices to coordinates

        # Flatten the coordinates and create a GeoDataFrame
        geometry = [Point(x[i], y[i]) for i in range(len(x.flatten()))]
        return gpd.GeoDataFrame(band_df, geometry=geometry, crs=crs)



In [12]:
import glob
plots=pd.concat([tiff_to_gdf(path) for path in glob.glob('/content/drive/MyDrive/Data/*tif')])

In [9]:
overall_median_approach_results=pd.read_csv('/content/drive/MyDrive/Data/regression_model_results_overall_median.csv')

In [8]:
import ast
import pandas as pd
import numpy as np
from xgboost import XGBRegressor

params=overall_median_approach_results.loc[(overall_median_approach_results['Model']=='XGBoost') & (overall_median_approach_results['Features']=='S2+DEM'),'Best Params']

params = ast.literal_eval(params.values[0])
Features=[ 'B2','B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12',
  'NBR', 'NDMI', 'NDRE','NDVI', 'TasseledCapGreenness',
  'elevation', 'slope']
target='rh95'

In [7]:
def exclude_outliers(data, target, min_percentile=0.05, max_percentile=99.5):
    min_threshold = np.percentile(data[target].to_numpy().astype(float), min_percentile)
    max_threshold = np.percentile(data[target].to_numpy().astype(float), max_percentile)
    print('min_threshold:', min_threshold)
    print('max_threshold:', max_threshold)
    return data[(data[target] >= min_threshold) & (data[target] <= max_threshold)]

data=pd.read_csv('/content/drive/MyDrive/Data/GEDI_S1_S2_DEM_Overall_Median.csv').dropna()
data = exclude_outliers(data,target,min_percentile=0.05, max_percentile=99)

min_threshold: 1.9800000190734863
max_threshold: 36.290001


In [10]:
model=XGBRegressor(**params,random_state=15)
model.fit(data[Features], data[target])


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=15, ...)

In [13]:
plots['CHM']=model.predict(plots[Features])

In [ ]:
plots.to_csv('/content/drive/MyDrive/Data/Predicted_CHM_FIA_Plots.csv')